# Creating a Water + Ion Waterbox and Performing a Quick Equilibration

This notebook walks you through **building a periodic box of water with 1 ion**, running an **energy minimisation**, and carrying out a short **NPT equilibration** with **LAMMPS**.  
By the end you will have:

* `waterbox_ion_initial.lmp` – the as-built, un-equilibrated system  
* `equil_waterbox.lmp` – the box after minimisation + equilibration  
* Thermodynamic logs (`log.waterbox`, `log.equil`) and a density plot for quick inspection.


<!-- Cell 1 -->
## How this notebook is organised

1. **Environment preparation** – load modules and point to your LAMMPS build  
2. **Create the template molecule** – look at `water.mol` and `ion.mol` and force-field parameters  
3. **Build the water box** – create `build_ion.in` and run it  
4. **Prepare minimisation & equilibration scripts** – write `equil_ion.in`  
5. **Run the equilibration** – launch LAMMPS with MPI  
6. **Analyse results** – plot density vs. time


<!-- Cell 4 -->
### 1 · Inspect the water-molecule template – what’s inside?

`water.mol` contains a **rigid** with atom IDs, coordinates, and topology in **LAMMPS “molecule” format**.  
`tip4pEw.param` holds the **TIP4P-Ew charges, LJ terms, and harmonic OH/HOH parameters** that will be included later.


<!-- Cell 5 (code follows) -->
#### Create `water.mol`


In [ ]:
from pathlib import Path

water_mol = r'''
# Water molecule

3 atoms
2 bonds
1 angles

Coords

1    0.00000  -0.06556   0.00000
2    0.75695   0.52032   0.00000
3   -0.75695   0.52032   0.00000

Types

1        1   # O
2        2   # H
3        2   # H

Charges

1       -0.834
2        0.417
3        0.417

Bonds

1   1      1      2
2   1      1      3

Angles

1   1      2      1      3
'''

Path('water.mol').write_text(water_mol)
print('water.mol is created.')

<!-- Cell 5 (code follows) -->
#### Create `ion.mol`


In [ ]:
from pathlib import Path

ion_mol = r'''
# anion

1 atoms

Coords

1    0.00000  0.000   0.00000

Types

1        1   # ion 

Charges

1        1
'''

Path('ion.mol').write_text(ion_mol)
print('ion.mol is created.')

<!-- Cell 6 (code follows) -->
#### Create `tip4pEw.param`

Similarly, this cell writes the full TIP4P-Ew parameter block, including:

* `pair_style lj/cut/tip4p/long` with the M-site offset  
* LJ coefficients for O–O and H–H  
* Grouping all O/H atoms into `h2o`


In [ ]:
from pathlib import Path

tip4pEw = r'''
## H2O ## TIP4P-Ew
set             type ${O} charge  -1.04844
set             type ${H} charge   0.52422

bond_coeff      ${OH_bond} 5000.0  0.9572
angle_coeff     ${OH_angle} 5000.0  104.52

variable OM_dist equal 0.1250

# pair style
pair_style &
  lj/cut/tip4p/long ${O} ${H} ${OH_bond} ${OH_angle} ${OM_dist} 9.0 9.0

# this is for soft pair style
# if you are not using soft, just remove the last column 
pair_coeff      ${O} ${O}  0.162750     3.16435
pair_coeff      ${H} ${H}  0.000000     1.0000   
pair_coeff      ${H}  *    0.000000     1.0000   

group h2o type ${O} ${H}
'''

Path('tip4pEw.param').write_text(tip4pEw)
print('tip4pEw.param is created.')

<!-- Cell 8 (code follows) -->
#### Generate `build_ion.in`

Key points:

* Calculates `box_dim` analytically from `num_water` and target `density`  
* Uses `create_atoms … mol water` to place molecules at random, allowing small overlaps  
* Includes the parameter file but **defers LJ mixing & k-space settings** to keep the file lightweight  
* Writes `waterbox_ion_initial.lmp` without coefficients (`nocoeff`)


In [ ]:
from pathlib import Path

build_in = r"""
#############################################
#  build_waterbox_ion.in
#  – Create a water box with a single monatomic ion and write it
#############################################

# Variables
variable     num_water      equal 256        # number of water molecules
variable     num_ion        equal 1          # number of ions (charge-balanced)
variable     density        equal 0.70       # g cm-3 (loose – will relax later)

# Calculate cubic box dimension (Å) from number of waters, density, and Avogadro's number
# box_dim = [ (mass_total / density) ]^(1/3)
# where mass_total ≃ num_water * molar_mass_water / N_A  (ion mass contribution is negligible)
variable     box_dim        equal (180*${num_water}/(6.022*${density}))^(1/3)

# System definition -----------------------------------------------------------
processors * * * map xyz
units      real
atom_style full
bond_style harmonic
angle_style harmonic
boundary p p p

# Define a cubic simulation region of size box_dim
region box block 0 ${box_dim} 0 ${box_dim} 0 ${box_dim}
# Create simulation box for **three** atom types (O, H, Ion)
create_box 3 box bond/types 1 angle/types 1 &
           extra/bond/per/atom 2 extra/angle/per/atom 1 extra/special/per/atom 2

# Masses (read by data file, but needed for molecule definition)

mass 1 22.990   # Na⁺
mass 2 15.9994
mass 3 1.008

# Convenience variables for molecule definitions
variable ion      equal 1
variable O        equal 2
variable H        equal 3
variable OH_bond  equal 1
variable OH_angle equal 1

# --- Molecule templates ------------------------------------------------------
molecule water water.mol
molecule ion   ion.mol

# --- Populate the box --------------------------------------------------------
# Randomly insert the ion; use a different seed to avoid correlation
create_atoms 0 random ${num_ion} 12345 NULL mol ion 24345 overlap 1.33
create_atoms 1 random ${num_water} 34564 NULL mol water 25678 overlap 1.33

# --- Interactions ------------------------------------------------------------
bond_style   harmonic
angle_style  harmonic
include      tip4pEw.param         # TIP4P-Ew coefficients (O/H)

# pair coeff for cation and anion <- change this based on your cation and anion
pair_coeff 1 1   0.1684375   2.184 # Na+ JC parameters

# --- K‑space & output --------------------------------------------------------
pair_modify mix arithmetic tail yes
kspace_style pppm/tip4p 1.0e-5

# Write the box (no coeffs so the second script can include updated ones)
write_data waterbox_ion_initial.lmp nocoeff
"""
Path('build_ion.in').write_text(build_in)
print('build_ion.in has been created.')


<!-- Cell 10 (code follows) -->
#### Build the initial box with LAMMPS

Runs `lmp_mpi_mbx` in serial, logging output to `log.waterbox`.  
Expect this to finish in a few seconds because no dynamics are performed yet.


In [ ]:
%%bash --login

module purge
module load shared slurm/expanse/23.02.7 sdsc/1.0 DefaultModules slurm/expanse/23.02.7 
module load cpu/0.17.3b intel/19.1.3.304/6pv46so intel-tbb/2020.3/lfesfxm intel-mpi/2019.10.317/ezrfjne fftw/3.3.10/tqkvj37

export LAMMPS_EXE=/expanse/projects/qstore/csd973/bin/lmp_mpi_mbx
"$LAMMPS_EXE"  -in build_ion.in  -log log.waterbox

<!-- Cell 11 -->
### 4 · Prepare minimisation & equilibration inputs – overview

We’ll minimise potential energy and then equilibrate at **298 K, 1 atm** for **10 ps** (10 000 × 1 fs).  
A fresh `equil_ion.in` is created so the workflow remains reproducible.


<!-- Cell 12 (code follows) -->
#### Generate `equil_ion.in`

Highlights:

* Defines temperature, timestep, pressure, and thermo print frequency  
* Reads `waterbox_ion_initial.lmp` and re-applies `tip4pEw.param`  
* Performs a **steepest-descent minimisation** (`minimize 1.0e-4 1.0e-6 500 2000`)  
* Switches to **NPT** with SHAKE constraints for rigid water geometry  
* Writes the equilibrated structure to `equil_waterbox.lmp`


In [ ]:
from pathlib import Path

equil_in = r"""
############################################################
#  equilibrate_waterbox_ion.in
#  Minimise and NPT‑equilibrate the pre‑built water‑plus‑ion box
############################################################

# Variables
variable     temp           equal 298.0      # K
variable     dt             equal 1.0        # fs
variable     P              equal 1          # atm
variable     thermo_freq    equal 100        # steps

variable ION      equal 1
variable O        equal 2
variable H        equal 3

variable OH_bond  equal 1
variable OH_angle equal 1

# System initialisation
processors * * * map xyz
units      real
atom_style full

# Read the pre‑built coordinates
read_data  waterbox_ion_initial.lmp

bond_style   harmonic
angle_style  harmonic
include      tip4pEw.param         # TIP4P‑Ew for water

# pair coeff for cation and anion <- change this based on your cation and anion
pair_coeff 1 1   0.1684375   2.184 # Na+ JC parameters

pair_modify  mix arithmetic tail yes
kspace_style pppm/tip4p 1.0e-5

# Neighbor list settings
neighbor 2.0 bin
neigh_modify every 1 delay 10 check yes

timestep ${dt}
thermo_style custom step time temp etotal press density
thermo ${thermo_freq}

# --- Energy minimisation then NPT -----
minimize 1.0e-4 1.0e-6 500 2000

velocity all create ${temp} 938478 mom yes rot yes dist gaussian

fix SHAKE all shake 1e-5 50 0 b ${OH_bond} a ${OH_angle}
fix NPT  all npt temp ${temp} ${temp} $(100.0*dt)  iso ${P} ${P} $(1000.0*dt)

# Run for 25 ps
run 10000

write_data equil_waterbox_ion.lmp
"""
Path('equil_ion.in').write_text(equil_in)
print('equil_ion.in has been created.')


<!-- Cell 13 -->
##### Running the equilibration

Depending on CPU count, the 20 ps run should take **~1–2 minutes**.


In [ ]:
sub_sh_script = r"""#!/bin/bash

#SBATCH --job-name="equil"
#SBATCH --output="equil.out"
#SBATCH --partition=debug
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=16
#SBATCH --mem=32GB
#SBATCH -A csd973
#SBATCH --export=ALL
#SBATCH -t 00:10:00

module purge
module load shared slurm/expanse/23.02.7 sdsc/1.0 DefaultModules slurm/expanse/23.02.7 cpu/0.17.3b intel/19.1.3.304/6pv46so intel-tbb/2020.3/lfesfxm intel-mpi/2019.10.317/ezrfjne fftw/3.3.10/tqkvj37

lammps=/expanse/projects/qstore/csd973/bin/lmp_mpi_mbx

export OMP_NUM_THREADS=16

$lammps -in equil_ion.in  -log log.equil
"""

with open('sub.sh', 'w') as f:
    f.write(sub_sh_script)

In [ ]:
!sbatch sub.sh

In [ ]:
!squeue --me

In [ ]:
!tail equil.out

<!-- Cell 15 -->
### 6 · Checking the density

A quick plot of density vs. simulation time lets us judge whether:

* The system has stabilised near **1 g cm⁻³**  


In [ ]:
import pandas as pd, re, pathlib, matplotlib.pyplot as plt

def read_lammps_log(path):
    """Return one tidy DataFrame with all thermo data in *every* block."""
    num_re  = re.compile(r"[+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?")
    blocks, header, rows = [], None, []

    for line in pathlib.Path(path).read_text().splitlines():
        txt = line.lstrip()

        if txt.startswith("Step"):
            if header and rows:                          # flush previous block
                blocks.append(pd.DataFrame(rows, columns=header))
                rows = []
            header = txt.split()
            continue

        if txt.startswith("Loop time of") and header:
            if rows:
                blocks.append(pd.DataFrame(rows, columns=header))
            header, rows = None, []
            continue

        if header:
            nums = num_re.findall(txt)                   # numeric tokens only
            if len(nums) == len(header):                 # ignore warnings etc.
                rows.append([float(x) for x in nums])

    # flush last block (if file didn’t end with “Loop time of”)
    if header and rows:
        blocks.append(pd.DataFrame(rows, columns=header))

    if not blocks:
        raise ValueError("No thermo blocks found in log file!")

    df = pd.concat(blocks, ignore_index=True)

    # If a header was duplicated inside a block (rare), drop the extras
    df = df.loc[:, ~df.columns.duplicated(keep="first")]

    return df

# ---------------- read + plot -------------------------------------------------
df = read_lammps_log("log.equil")
df["Time_ps"] = df["Step"] * 1e-3      # 1 fs → 0.001 ps

plt.figure(figsize=(6, 4))
plt.plot(df["Time_ps"].to_numpy(), df["Density"].to_numpy())
plt.title("Density vs. Time")
plt.xlabel("Time (ps)")
plt.ylabel(r"Density (g cm$^{-3}$)")
plt.tight_layout()
plt.show()

<!-- Cell 17 -->
### What to expect

After ~2 ps the density should plateau near **1 g cm⁻³**; small fluctuations are normal.  
We will perform actual equlibration and produciton run using MB-pol potential in the next step.
